In [1]:
import sys, os
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import pygraphviz as pgv
from pygraphviz import *
import numpy as np
import plotly.graph_objects as go
import tqdm

In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
try:
    from nltk.corpus import wordnet
except:
    import nltk
    nltk.download('wordnet')
from nltk.corpus import wordnet

In [4]:
def ellipse(x_center=0, y_center=0, ax1=[1, 0], ax2=[0, 1], a=1, b=1, N=100):
    # x_center, y_center the coordinates of ellipse center
    # ax1 ax2 two orthonormal vectors representing the ellipse axis directions
    # a, b the ellipse parameters
    if np.linalg.norm(ax1) != 1 or np.linalg.norm(ax2) != 1:
        raise ValueError('ax1, ax2 must be unit vectors')
    if abs(np.dot(ax1, ax2)) > 1e-06:
        raise ValueError('ax1, ax2 must be orthogonal vectors')
    t = np.linspace(0, 2 * np.pi, N)
    # ellipse parameterization with respect to a system of axes of directions a1, a2
    xs = a * np.cos(t)
    ys = b * np.sin(t)
    # rotation matrix
    R = np.array([ax1, ax2]).T
    # coordinate of the  ellipse points with respect to the system of axes [1, 0], [0,1] with origin (0,0)
    xp, yp = np.dot(R, [xs, ys])
    x = xp + x_center
    y = yp + y_center
    return x, y

In [10]:
def isindict(w, d):
    for key in d.keys():
        for w2 in d[key]:
            if w.strip() == w2.strip():
                return key
def addtodict(w1,w2,d):
    w1_key = isindict(w1, groups_dict)
    w2_key = isindict(w2, groups_dict)
    
    
    if w1_key == w2_key == None:
        group_name = 'n'+str(len(groups_dict))
        groups_dict[group_name] = set([w1, w2])
        return
    if w1_key is not None:
        groups_dict[w1_key].add(w2)
    if w2_key is not None:
        groups_dict[w2_key].add(w1)           
                

In [11]:
df = pd.read_csv("word_data_detailed.csv", engine='python')
df = df.fillna('')
df['word'] = df['word'].map(lambda x: x.strip())
df = df.set_index('word')


In [12]:
groups = open("groups.txt")
groups_dict = {}
for line in groups.readlines():
    group_name = 'n'+str(len(groups_dict))
    
    groups_dict[group_name] = set()
    for word in line.strip().split():
        groups_dict[group_name].add(word)

# refine groups
current_w = 0
for w1 in tqdm.tqdm(df.index):
    
    # word similarity
    for w2 in df.index[current_w+1:]:
        sw1 = wordnet.synsets(w1)
        sw2 = wordnet.synsets(w2)
        sim = 0
        for _w1 in sw1:
            for _w2 in sw2:
                 sim = max(sim,_w1.wup_similarity(_w2))
        if sim > 0.9:
            addtodict(w1,w2,groups_dict)
    current_w += 1
    
# for w in tqdm.tqdm(df.index):
#     for s in df.loc[w]['synonyms'].split(','):
#         s = s.strip()
#         gs_key = isindict(s, groups_dict)
        
#         if gs_key is not None: # s in group. add w to that group
#             groups_dict[gs_key].add(w)
    

100%|████████████████████████████████████████████████████████████████████████████████| 808/808 [03:13<00:00,  4.18it/s]


In [336]:
for w in wordnet.synsets('abound'):
    print(w.definition())

be abundant or plentiful; exist in large quantities
be in a state of movement or action


In [13]:
groups_dict

{'n0': {'exacting',
  'exigent',
  'fastidious',
  'meticulous',
  'painstaking',
  'pedantic',
  'polemical',
  'pristine',
  'punctilious',
  'scrupulous'},
 'n1': {'congenial', 'convivial'},
 'n2': {'accentuate',
  'conspicuous',
  'disparate',
  'flamboyant',
  'florid',
  'sanguine',
  'underscore'},
 'n3': {'banal',
  'hackneyed',
  'humdrum',
  'monotonous',
  'mundane',
  'pedestrian',
  'plodding',
  'prosaic',
  'soporific'},
 'n4': {'innocuous', 'insipid', 'vapid'},
 'n5': {'cacophonous',
  'clamorous',
  'garrulous',
  'loquacious',
  'pugnacious',
  'truculent',
  'vociferous'},
 'n6': {'extravagant',
  'libertine',
  'parsimonious',
  'prodigal',
  'profligate',
  'spendthrift'},
 'n7': {'aloof', 'diffident', 'distant', 'taciturn', 'timorous'},
 'n8': {'circumspect', 'discreet', 'wary'},
 'n9': {'byzantine', 'convoluted', 'sophisticated', 'tortuous'},
 'n10': {'cursory', 'perfunctory', 'superficial', 'trivial'},
 'n11': {'adulterate', 'impair', 'subvert', 'vitiate'},
 'n1

In [14]:
g = pgv.AGraph()
for w in df.index:
    g.add_node(w, label=w)        
# for key in groups_dict:
#     g.add_node(key, label=key)
#     for w in groups_dict[key]:
#         g.add_edge(w, key)
for key in groups_dict:
    li = list(groups_dict[key])
    for x in range(len(li)-1):
        g.add_edge(li[x], li[x+1])
for n in g.nodes():
    if n.attr['label'] == '\\N':
        print(n)
g.layout()

In [15]:
df

,meaning,synonyms,antonyms,examples,definitions
word,,,,,
abound,large numbers or amounts,"abound,bristle,burst",,1. The room abounded with screaming children \...,be abundant or plentiful; exist in large quant...
amorphous,without a clearly defined shape or form,"amorphous,uncrystallized,uncrystallised,unstru...",,1. amorphous clouds of insects \n2. an aggrega...,having no definite form or distinct shape \nla...
austere,"strict in manner, having no comforts or luxuries","spartan,austere,stark,ascetical,ascetic,severe...",,1. a stark interior \n1. an austere expression...,severely simple \nof a stern or strict bearing...
belie,"fail to give true impression of something, con...","negate,contradict,misrepresent,belie",,1. This statement misrepresents my intentions \n,be in contradiction with \nrepresent falsely \n
capricious,"sudden change of mood, unpredictable, fickle","capricious,whimsical,freakish,impulsive",,1. a capricious summer breeze \n2. freakish we...,changeable \ndetermined by chance or impulse o...
...,...,...,...,...,...
squander,waste in a reckless manner,"waste,squander,consume,ware,blow",conserve,1. He wasted his inheritance on his insincere ...,spend thoughtlessly; throw away \nspend extrav...
cajole,persuade to do something by flattery,"wheedle,coax,sweet-talk,cajole,inveigle,blarne...",,1. He palavered her into going along \n,"influence or urge by gentle urging, caressing,..."
unalloyed,pure/ complete and unreserved,unalloyed,,1. unalloyed metal \n2. unalloyed pleasure \n,free from admixture \n


In [16]:
layout = []
for n in g.nodes():
    w = g.get_node(n).attr['label'].strip()
    tmp_attr = {'label': w}
    tmp_attr['x'] = float(g.get_node(n).attr['pos'].split(",")[0])
    tmp_attr['y'] = float(g.get_node(n).attr['pos'].split(",")[1])
    tmp_attr['height'] = float(g.get_node(n).attr['height'])
    tmp_attr['width'] = float(g.get_node(n).attr['width'])

    if tmp_attr['label'] in df.index.values:
        tmp_attr['hover'] = "meaning#1: " + df.loc[w]['meaning']
        tmp_attr['hover'] += "\nmeaning#2: " + df.loc[w]['definitions']
        tmp_attr['hover'] += "\nsynonyms: " + df.loc[w]['synonyms']
        tmp_attr['hover'] += "\nantonyms: " + df.loc[w]['antonyms']
        tmp_attr['hover'] += "\nexamples: " + df.loc[w]['examples']
    else:
        tmp_attr['hover'] = ""
    layout.append(tmp_attr)
df2 = pd.DataFrame.from_dict(layout)
df2['label'] = df2['label'].astype(str)
df2 =df2.set_index('label')
g.draw("file.png")

In [17]:
df3 = df.join(df2, how='outer')

In [18]:
fig = go.Figure()
a=0
b=len(df3)
df3['examples'] = df3['examples'].fillna('').map(lambda x: x.replace('\n','<br>'))
fig.add_trace(go.Scatter(
    x=df3['x'].values[a:b],
    y=df3['y'].values[a:b],
    mode="text",
    text=list(df3.index.values[a:b]),
    customdata=df3[['meaning', 'definitions', 'synonyms','antonyms','examples']].values,
#     hovertext=df3['hover'],
    hovertemplate = "%{text}<br>meaning#1:%{customdata[0]}<br>meaning#2:%{customdata[1]}<br>synonyms:%{customdata[2]}<br>antonyms:%{customdata[3]}<br>examples:<br>%{customdata[4]}",
    textfont=dict(
        family="sans serif",
        size=14,
        color="black"
    ),
    showlegend=False,
))
# fig.update_traces(textposition='middle center', textfont_size=14)
# fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

scale=50
for idx, r in df3.iterrows():
    x, y = ellipse(r['x'], r['y'], a=scale*r['width'] / 2, b=scale*r['height'] )
    fig.add_scatter(x=x, y=y, mode='lines', hoverinfo='skip',showlegend=False)
for i in range(len(g.edges())):
    node1, node2 = g.get_node(g.edges()[i][0]),g.get_node(g.edges()[i][1])
    tmp_edge = np.array([list(map(float,node1.attr['pos'].split(','))), list(map(float,node2.attr['pos'].split(',')))])
    
#     tmp_edge = g.edges()[i].attr['pos'].split(" ")
# #     tmp_edge =[tmp_edge[0]] + [tmp_edge[3]]
#     tmp_edge = np.array([x.split(',') for x in tmp_edge])
    fig.add_scatter(x=tmp_edge[:,0], y=tmp_edge[:,1], mode='lines', hoverinfo='skip', showlegend=False)
fig.write_html('output.html', auto_open=True )
# fig.show()

In [19]:
df3[df3.index=='\\N']

,meaning,synonyms,antonyms,examples,definitions,x,y,height,width,hover


## (Optional) Add details to word list

In [326]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()
print (dictionary.meaning("indentation"))
# dictionary.synonym("everyone") # mow working

{'Noun': ['a concave cut into a surface or edge (as in a coastline', 'the formation of small pits in a surface as a consequence of corrosion', 'the space left between the margin and the start of an indented line', 'the act of cutting into an edge with toothlike notches or angular incisions']}


In [327]:
_df = pd.read_csv("word_data.csv", engine='python')
_df['word'] = _df['word'].map(lambda x:x.strip())
_df

,word,meaning,synonyms
0,abound,large numbers or amounts,NaN
1,amorphous,without a clearly defined shape or form,NaN
2,austere,"strict in manner, having no comforts or luxuries",NaN
3,belie,"fail to give true impression of something, con...",NaN
4,capricious,"sudden change of mood, unpredictable, fickle",NaN
...,...,...,...
803,squander,waste in a reckless manner,NaN
804,cajole,persuade to do something by flattery,NaN
805,unalloyed,pure/ complete and unreserved,NaN
806,heady,intoxicating / having a strong or exhilarating...,NaN


In [329]:
synonyms = []
antonyms = []
examples = []
definitions = []
for word in _df['word']:
    word = word.strip()
    syns = wordnet.synsets(word)
#     print(word, end=' ')
#     print(syns[0].lemmas()[0].name())
    _str_examples = ""
    _str_definitions = ""
    for sy in syns:
        _examples = sy.examples()
        for idx,ex in enumerate(_examples):
            _str_examples += str(idx+1)+". "+ex+" \n"
        
        _str_definitions += sy.definition()+" \n"
    if len(_str_examples) == 0:
            _str_examples += "None\n"
    
    examples.append(_str_examples)
    definitions.append(_str_definitions)
    
    _synonyms = []
    _antonyms = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            _synonyms.append(l.name())
            if l.antonyms():
                _antonyms.append(l.antonyms()[0].name())
    synonyms.append(','.join(set(_synonyms)))
    antonyms.append(','.join(set(_antonyms)))
_df['synonyms'] = synonyms
_df['antonyms'] = antonyms
_df['examples'] = examples
_df['definitions'] = definitions


In [330]:
_df

,word,meaning,synonyms,antonyms,examples,definitions
0,abound,large numbers or amounts,"abound,bristle,burst",,1. The room abounded with screaming children \...,be abundant or plentiful; exist in large quant...
1,amorphous,without a clearly defined shape or form,"amorphous,uncrystallized,uncrystallised,unstru...",,1. amorphous clouds of insects \n2. an aggrega...,having no definite form or distinct shape \nla...
2,austere,"strict in manner, having no comforts or luxuries","spartan,austere,stark,ascetical,ascetic,severe...",,1. a stark interior \n1. an austere expression...,severely simple \nof a stern or strict bearing...
3,belie,"fail to give true impression of something, con...","negate,contradict,misrepresent,belie",,1. This statement misrepresents my intentions \n,be in contradiction with \nrepresent falsely \n
4,capricious,"sudden change of mood, unpredictable, fickle","capricious,whimsical,freakish,impulsive",,1. a capricious summer breeze \n2. freakish we...,changeable \ndetermined by chance or impulse o...
...,...,...,...,...,...,...
803,squander,waste in a reckless manner,"waste,squander,consume,ware,blow",conserve,1. He wasted his inheritance on his insincere ...,spend thoughtlessly; throw away \nspend extrav...
804,cajole,persuade to do something by flattery,"wheedle,coax,sweet-talk,cajole,inveigle,blarne...",,1. He palavered her into going along \n,"influence or urge by gentle urging, caressing,..."
805,unalloyed,pure/ complete and unreserved,unalloyed,,1. unalloyed metal \n2. unalloyed pleasure \n,free from admixture \n
806,heady,intoxicating / having a strong or exhilarating...,"foolhardy,intoxicating,wise,rash,heady,judicio...",,1. judicious use of one's money \n2. a wise de...,marked by the exercise of good judgment or com...


In [331]:
_df.to_csv('word_data_detailed.csv', index=False)